# How to make a thread-safe generator

Short answer is you cannot.

* [Using iterators and generators in multi-threaded applications](https://anandology.com/blog/using-iterators-and-generators/)

> if two threads try to call next method on a generator at the same time, it will raise an exception ```ValueError: generator already executing```. The only way to fix it is by wrapping it in an iterator and have a lock that allows only one thread to call next method of the generator.

* [Are Generators Threadsafe?](https://stackoverflow.com/questions/1131430/are-generators-threadsafe)
* [Python builtin - iter](https://docs.python.org/3/library/functions.html#iter)

> #### iter(object)
> ```object``` must be a collection object which supports the iterable protocol (the __iter__() method), or it must support the sequence protocol (the ```__getitem__()``` method with integer arguments starting at 0). If it does not support either of those protocols, TypeError is raised.

In [1]:
import threading
import functools

In [2]:
def count():
    i = 0
    while True:
        i += 1
        yield i

class Counter:
    def __init__(self):
        self.i = 0

    def __iter__(self):
        return self

    def __next__(self):
        self.i += 1
        return self.i

def loop(generator, n):
    """Runs the given function n times in a loop.
    """
    for i in range(n):
        next(generator)

def run(f, repeats=1000, nthreads=10):
    """Starts multiple threads to execute the given function multiple
    times in each thread.
    """
    # create threads
    threads = [threading.Thread(target=loop, args=(f, repeats)) 
               for i in range(nthreads)]

    # start threads
    for t in threads:
        t.start()

    # wait for threads to finish
    for t in threads:
        t.join()

def main():
    c1 = count()
    c2 = Counter()

    # call c1.next 100K times in 2 different threads
    run(c1, repeats=100000, nthreads=2)
    print("c1", next(c1))

    # call c2.next 100K times in 2 different threads
    run(c2, repeats=100000, nthreads=2)
    print("c2", next(c2))

In [8]:
# This may succeeds by chance. Try multiple times to get "ValueError: generator already executing"
main()

Exception in thread Thread-26:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/_4/8v285hqs45xfzk0l1nlr3yq40000gn/T/ipykernel_49390/4243573237.py", line 22, in loop
ValueError: generator already executing


c1 100001
c2 200001


# Solution

In [6]:
class ThreadSafeIterator:
    """Wrapper to make a generator thread-safe
    See
        * https://docs.python.org/3/library/functions.html#iter
        * https://anandology.com/blog/using-iterators-and-generators/
    """
    def __init__(self, iterable):
        self.lock = threading.Lock()
        self.iterable = iter(iterable)

    def __iter__(self):
        return self

    def __next__(self):
        with self.lock:
            return self.iterable.__next__()

In [11]:
def threadsafe_iterator(f):
    """A decorator that makes an iterable thread-safe.
    """
    @functools.wraps(f)
    def g(*args, **kwargs):
        return ThreadSafeIterator(f(*args, **kwargs))
    return g


In [15]:
@threadsafe_iterator
def count2():
    i = 0
    while True:
        i += 1
        yield i

@threadsafe_iterator
class Counter2:
    def __init__(self):
        self.i = 0

    def __iter__(self):
        return self

    def __next__(self):
        self.i += 1
        return self.i

In [18]:
def main2():
    c1_2 = count2()
    c2_2 = Counter2()

    # call 100K times in 2 different threads
    run(c1_2, repeats=100000, nthreads=2)
    print("c1_2", next(c1_2))

    # call 100K times in 2 different threads
    run(c2_2, repeats=100000, nthreads=2)
    print("c2_2", next(c2_2))

In [20]:
main2()

c1_2 200001
c2_2 200001
